In [1]:
from project.utils import get_pandas
import pandas as pd

In [164]:
profeel = get_pandas('project/input/stock/building_stock_profeel_vent.csv', lambda x: pd.read_csv(x, index_col=[0, 1]).squeeze())
sdes = get_pandas('project/input/stock/building_stock_sdes2018_aggregated.csv', lambda x: pd.read_csv(x, index_col=[0, 1]).squeeze())

In [165]:
profeel = profeel.rename(columns={'calculated_epc':'Energy performance'})
profeel['Heating energy'] = profeel['Heating system'].str.split('-').str[0]
profeel = profeel.set_index(['Heating energy', 'Energy performance'], append=True)
profeel

,,,,Heating system,Wall,Floor,Roof,Windows,Ventilation,Stock buildings,Efficiency,energy_primary
Class,Housing type,Heating energy,Energy performance,,,,,,,,,
MI-1,Single-family,Wood fuel,F,Wood fuel-Standard boiler,2.50,2.00,2.50,2.4,0.4,688400,0.75,423.726894
MI-2,Single-family,Wood fuel,F,Wood fuel-Standard boiler,2.50,2.00,2.50,2.4,0.4,648300,0.75,408.608469
MI-3,Single-family,Wood fuel,G,Wood fuel-Standard boiler,2.50,2.00,2.50,2.4,0.4,622300,0.75,461.921928
MI-4,Single-family,Wood fuel,F,Wood fuel-Standard boiler,2.50,2.00,2.50,2.4,0.4,3186600,0.75,368.010932
MI-5,Single-family,Natural gas,G,Natural gas-Standard boiler,2.50,2.00,2.50,2.4,0.4,612000,0.60,524.692442
MI-6,Single-family,Natural gas,F,Natural gas-Standard boiler,2.50,2.00,2.50,2.4,0.4,86300,0.60,394.481757
MI-7,Single-family,Natural gas,F,Natural gas-Standard boiler,2.50,2.00,2.50,2.4,0.4,741300,0.60,368.717101
MI-8,Single-family,Natural gas,G,Natural gas-Standard boiler,2.50,2.00,2.50,2.4,0.4,1364000,0.60,497.531662
MI-9,Single-family,Natural gas,F,Natural gas-Standard boiler,2.50,2.00,2.50,2.4,0.4,923400,0.60,416.055541


In [166]:
sdes = sdes.set_index(['Heating energy', 'Occupancy status', 'Income owner', 'Income tenant'], append=True)
sdes

Stock buildings
Housing type  Energy performance Heating energy Occupancy status Income owner Income tenant                 
Multi-family  A                  Electricity    Social-housing   D10          D1                 4162.321113
Single-family A                  Electricity    Social-housing   D10          D1                 1618.652411
Multi-family  A                  Electricity    Owner-occupied   D1           D1                 1154.718085
Single-family A                  Electricity    Owner-occupied   D1           D1                 5032.777538
Multi-family  A                  Electricity    Social-housing   D10          D2                 4298.192125
...                                                                                                      ...
Single-family G                  Wood fuel      Privately rented D5           D10                  20.603479
                                                                 D6           D10                  25.586547
                                                                 D7           D10                  34.844713
                                                                 D8           D10                  63.545738
                                                                 D9           D10                 110.578082

[6720 rows x 1 columns]

In [168]:
#sdes = sdes.reset_index(level = ['Occupancy status', 'Income owner', 'Income tenant'])
merged = profeel.merge(sdes, how='inner', on=['Housing type', 'Energy performance', 'Heating energy'])
merged

Heating system  \
Housing type  Energy performance Heating energy                                   
Single-family F                  Wood fuel            Wood fuel-Standard boiler   
                                 Wood fuel            Wood fuel-Standard boiler   
                                 Wood fuel            Wood fuel-Standard boiler   
                                 Wood fuel            Wood fuel-Standard boiler   
                                 Wood fuel            Wood fuel-Standard boiler   
...                                                                         ...   
Multi-family  B                  Electricity     Electricity-Performance boiler   
                                 Electricity     Electricity-Performance boiler   
                                 Electricity     Electricity-Performance boiler   
                                 Electricity     Electricity-Performance boiler   
                                 Electricity     Electricity-Performance boiler   

                                                 Wall  Floor  Roof  Windows  \
Housing type  Energy performance Heating energy                               
Single-family F                  Wood fuel       2.50   2.00   2.5      2.4   
                                 Wood fuel       2.50   2.00   2.5      2.4   
                                 Wood fuel       2.50   2.00   2.5      2.4   
                                 Wood fuel       2.50   2.00   2.5      2.4   
                                 Wood fuel       2.50   2.00   2.5      2.4   
...                                               ...    ...   ...      ...   
Multi-family  B                  Electricity     0.34   0.25   0.2      1.6   
                                 Electricity     0.34   0.25   0.2      1.6   
                                 Electricity     0.34   0.25   0.2      1.6   
                                 Electricity     0.34   0.25   0.2      1.6   
                                 Electricity     0.34   0.25   0.2      1.6   

                                                 Ventilation  \
Housing type  Energy performance Heating energy                
Single-family F                  Wood fuel               0.4   
                                 Wood fuel               0.4   
                                 Wood fuel               0.4   
                                 Wood fuel               0.4   
                                 Wood fuel               0.4   
...                                                      ...   
Multi-family  B                  Electricity             0.2   
                                 Electricity             0.2   
                                 Electricity             0.2   
                                 Electricity             0.2   
                                 Electricity             0.2   

                                                 Stock buildings_x  \
Housing type  Energy performance Heating energy                      
Single-family F                  Wood fuel                  688400   
                                 Wood fuel                  688400   
                                 Wood fuel                  688400   
                                 Wood fuel                  688400   
                                 Wood fuel                  688400   
...                                                            ...   
Multi-family  B                  Electricity               2641000   
                                 Electricity               2641000   
                                 Electricity               2641000   
                                 Electricity               2641000   
                                 Electricity               2641000   

                                                 Efficiency  energy_primary  \
Housing type  Energy performance Heating energy                               
Single-family F                  Wood fuel             

In [142]:
index = pd.Series(range(0,4680))
index
merged.set_index(index, append=True)

Heating system  \
Housing type  Energy performance Heating energy                                        
Single-family F                  Wood fuel      0          Wood fuel-Standard boiler   
                                                1          Wood fuel-Standard boiler   
                                                2          Wood fuel-Standard boiler   
                                                3          Wood fuel-Standard boiler   
                                                4          Wood fuel-Standard boiler   
...                                                                              ...   
Multi-family  B                  Electricity    4675  Electricity-Performance boiler   
                                                4676  Electricity-Performance boiler   
                                                4677  Electricity-Performance boiler   
                                                4678  Electricity-Performance boiler   
                                                4679  Electricity-Performance boiler   

                                                      Wall  Floor  Roof  \
Housing type  Energy performance Heating energy                           
Single-family F                  Wood fuel      0     2.50   2.00   2.5   
                                                1     2.50   2.00   2.5   
                                                2     2.50   2.00   2.5   
                                                3     2.50   2.00   2.5   
                                                4     2.50   2.00   2.5   
...                                                    ...    ...   ...   
Multi-family  B                  Electricity    4675  0.34   0.25   0.2   
                                                4676  0.34   0.25   0.2   
                                                4677  0.34   0.25   0.2   
                                                4678  0.34   0.25   0.2   
                                                4679  0.34   0.25   0.2   

                                                      Windows  Ventilation  \
Housing type  Energy performance Heating energy                              
Single-family F                  Wood fuel      0         2.4          0.4   
                                                1         2.4          0.4   
                                                2         2.4          0.4   
                                                3         2.4          0.4   
                                                4         2.4          0.4   
...                                                       ...          ...   
Multi-family  B                  Electricity    4675      1.6          0.2   
                                                4676      1.6          0.2   
                                                4677      1.6          0.2   
                                                4678      1.6          0.2   
                                                4679      1.6          0.2   

                                                      Stock buildings_x  \
Housing type  Energy performance Heating energy                           
Single-family F                  Wood fuel      0                688400   
                                                1                688400   
                                                2                688400   
                                                3                688400   
                                                4                688400   
...                                                                 ...   
Multi-family  B                  Electricity    4675            2641000   
                                                4676            2641000   
                                                4677            2641000   
                                                4678            2641000   
                                  

In [175]:
merged = merged.set_index(['Occupancy status', 'Income owner', 'Income tenant'], append=True)
weights = (merged['Stock buildings_y'].T / merged.groupby(level=['Housing type', 'Energy performance', 'Heating energy'])['Stock buildings_y'].sum()).T
weights = weights.sort_index()
test = (weights.T * merged['Stock buildings_x'].sort_index()).T
test

Housing type   Energy performance  Heating energy  Occupancy status  Income owner  Income tenant
Multi-family   B                   Electricity     Owner-occupied    D1            D1                15648.149523
                                                                     D10           D10              145604.613178
                                                                     D2            D2                30312.341933
                                                                     D3            D3                39656.950882
                                                                     D4            D4                44546.079668
                                                                                                        ...      
Single-family  G                   Wood fuel       Social-housing    D10           D5                  455.088785
                                                                                   D6                  35

In [179]:
merged.groupby(level=['Housing type', 'Energy performance', 'Heating energy'])['Stock buildings_y'].sum()

Housing type   Energy performance  Heating energy  Income owner
Multi-family   B                   Electricity     D1               4215.854870
                                                   D10             87478.189504
                                                   D2               4320.225342
                                                   D3               5165.974022
                                                   D4               5888.642854
                                                                       ...     
Single-family  G                   Wood fuel       D5              12210.121994
                                                   D6              11557.377287
                                                   D7              11325.805510
                                                   D8              11211.601469
                                                   D9              10482.076433
Name: Stock buildings_y, Length: 200, dtype: float64

In [169]:
print("profeel building stock: {}".format(profeel['Stock buildings'].sum()))
print("sdes building stock: {}".format(sdes['Stock buildings'].sum()))
print("merged building stock: {}".format(merged['Stock buildings_x'].sum()/(merged.shape[0]/profeel.shape[0])))
print("output building stock: {}".format(test.sum()))

In [125]:
#Where does this come from?: 120, 20*3*2
merged.shape[0]/profeel.shape[0]

120.0

In [63]:
occup_prop = sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status']).sum()/sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy']).sum()
tenant_prop = sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status', 'Income owner']).sum()/sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status']).sum()
owner_prop = sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status', 'Income owner', 'Income tenant']).sum()/sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status', 'Income owner']).sum()

profeel.merge(sdes, how='outer', on=['Housing type', 'Energy performance', 'Heating energy'])

Heating system  \
Housing type  Energy performance Heating energy                              
Single-family F                  Wood fuel       Wood fuel-Standard boiler   
                                 Wood fuel       Wood fuel-Standard boiler   
                                 Wood fuel       Wood fuel-Standard boiler   
                                 Wood fuel       Wood fuel-Standard boiler   
                                 Wood fuel       Wood fuel-Standard boiler   
...                                                                    ...   
              G                  Oil fuel                              NaN   
                                 Oil fuel                              NaN   
                                 Oil fuel                              NaN   
                                 Oil fuel                              NaN   
                                 Oil fuel                              NaN   

                                                 Wall  Floor  Roof  Windows  \
Housing type  Energy performance Heating energy                               
Single-family F                  Wood fuel        2.5    2.0   2.5      2.4   
                                 Wood fuel        2.5    2.0   2.5      2.4   
                                 Wood fuel        2.5    2.0   2.5      2.4   
                                 Wood fuel        2.5    2.0   2.5      2.4   
                                 Wood fuel        2.5    2.0   2.5      2.4   
...                                               ...    ...   ...      ...   
              G                  Oil fuel         NaN    NaN   NaN      NaN   
                                 Oil fuel         NaN    NaN   NaN      NaN   
                                 Oil fuel         NaN    NaN   NaN      NaN   
                                 Oil fuel         NaN    NaN   NaN      NaN   
                                 Oil fuel         NaN    NaN   NaN      NaN   

                                                 Ventilation  \
Housing type  Energy performance Heating energy                
Single-family F                  Wood fuel               0.4   
                                 Wood fuel               0.4   
                                 Wood fuel               0.4   
                                 Wood fuel               0.4   
                                 Wood fuel               0.4   
...                                                      ...   
              G                  Oil fuel                NaN   
                                 Oil fuel                NaN   
                                 Oil fuel                NaN   
                                 Oil fuel                NaN   
                                 Oil fuel                NaN   

                                                 Stock buildings_x  \
Housing type  Energy performance Heating energy                      
Single-family F                  Wood fuel                688400.0   
                                 Wood fuel                688400.0   
                                 Wood fuel                688400.0   
                                 Wood fuel                688400.0   
                                 Wood fuel                688400.0   
...                                                            ...   
              G                  Oil fuel                      NaN   
                                 Oil fuel                      NaN   
                                 Oil fuel                      NaN   
                                 Oil fuel                      NaN   
                                 Oil fuel                      NaN   

                                                 Efficiency  energy_primary  \
Housing type  Energy performance Heating energy                               
Single-family F                  Wood fuel             0.75      423.726894   
                                 Woo

In [99]:
occup_prop = sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status']).sum()/sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy']).sum()
profeel = profeel.groupby(level=['Housing type', 'Energy performance', 'Heating energy'])['Stock buildings'].sum().rename('Stock buildings')
tenant_prop = sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status', 'Income owner']).sum()/sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status']).sum()
owner_prop = sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status', 'Income owner', 'Income tenant']).sum()/sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status', 'Income owner']).sum()
profeel_occup = occup_prop.multiply(profeel, axis=0).dropna()

In [108]:
profeel_occup

Stock buildings
Housing type  Energy performance Heating energy Occupancy status                 
Multi-family  B                  Electricity    Owner-occupied       7.558572e+05
                                                Privately rented     1.243852e+06
                                                Social-housing       6.412905e+05
              C                  Electricity    Owner-occupied       2.656255e+05
                                                Privately rented     3.689851e+05
                                                Social-housing       1.333895e+05
                                 Natural gas    Owner-occupied       2.565633e+05
                                                Privately rented     2.197016e+05
                                                Social-housing       3.600352e+05
              D                  Electricity    Owner-occupied       5.451666e+05
                                                Privately rented     9.907463e+05
                                                Social-housing       3.281870e+05
                                 Natural gas    Owner-occupied       6.136537e+05
                                                Privately rented     5.347581e+05
                                                Social-housing       7.070882e+05
              E                  Electricity    Owner-occupied       7.399848e+05
                                                Privately rented     1.664741e+06
                                                Social-housing       4.916743e+05
                                 Natural gas    Owner-occupied       7.785191e+05
                                                Privately rented     8.417635e+05
                                                Social-housing       6.101175e+05
              F                  Electricity    Owner-occupied       1.346884e+05
                                                Privately rented     3.943556e+05
                                                Social-housing       7.045594e+04
                                 Natural gas    Owner-occupied       4.927132e+05
                                                Privately rented     6.985192e+05
                                                Social-housing       3.875676e+05
              G                  Electricity    Owner-occupied       1.225168e+05
                                                Privately rented     4.738726e+05
                                                Social-housing       3.641059e+04
Single-family B                  Natural gas    Owner-occupied       1.507268e+06
                                                Privately rented     2.417782e+05
                                                Social-housing       3.701543e+05
              C                  Natural gas    Owner-occupied       7.958925e+05
                                                Privately rented     1.214964e+05
                                                Social-housing       1.359111e+05
              D                  Electricity    Owner-occupied       1.439418e+06
                                                Privately rented     3.289199e+05
                                                Social-housing       6.766259e+04
                                 Natural gas    Owner-occupied       4.801252e+05
                                                Privately rented     6.682596e+04
                                                Social-housing       5.664886e+04
              E                  Electricity    Owner-occupied       2.565609e+06
                                                Privately rented     5.837361e+05
                                                Social-housing       2.230552e+05
                                 Natural gas    Owner-occupied       3.594915e+05
                                                Privately rented     5.301836e+04
                                                

In [111]:
owner_prop = sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status', 'Income owner']).sum()/sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status']).sum()
profeel_occup_owner = owner_prop.multiply(profeel_occup, axis=0).dropna()

In [112]:
profeel_occup_owner

Stock buildings
Housing type  Energy performance Heating energy Occupancy status Income owner                 
Multi-family  B                  Electricity    Owner-occupied   D1               15648.149523
                                                                 D10             145604.613178
                                                                 D2               30312.341933
                                                                 D3               39656.950882
                                                                 D4               44546.079668
...                                                                                        ...
Single-family G                  Wood fuel      Privately rented D6                6807.165786
                                                                 D7                8264.703866
                                                                 D8               10634.455001
                                                                 D9               14262.287552
                                                Social-housing   D10               4592.889676

[420 rows x 1 columns]

In [113]:
tenant_prop = sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status',
                                 'Income owner', 'Income tenant']).sum() / sdes.groupby(
    level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status', 'Income owner']).sum()
profeel_occup_owner_tenant = tenant_prop.multiply(profeel_occup_owner, axis=0).dropna()

In [114]:
profeel_occup_owner_tenant

Stock buildings
Housing type  Energy performance Heating energy Occupancy status Income owner Income tenant                 
Multi-family  B                  Electricity    Owner-occupied   D1           D1                15648.149523
                                                                 D10          D10              145604.613178
                                                                 D2           D2                30312.341933
                                                                 D3           D3                39656.950882
                                                                 D4           D4                44546.079668
...                                                                                                      ...
Single-family G                  Wood fuel      Social-housing   D10          D5                  455.088785
                                                                              D6                  351.906208
                                                                              D7                  260.714629
                                                                              D8                  168.934239
                                                                              D9                   90.464177

[2400 rows x 1 columns]